In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy 
from cartopy.util import add_cyclic_point
import cartopy.mpl.ticker as cticker
import xesmf as xe
import matplotlib.colors as colors
from scipy.stats import pearsonr

In [2]:
def regrid(ds, var, lat_step=0.1, lon_step=0.1):
    # Create new lons and lats on a regular grid
    new_lat=np.arange(-90,90.1,lat_step)
    new_lon=np.arange(0,360.1,lon_step)

    # Create an xarray.Dataset that defines the new grid
    ds_out = xr.Dataset({'lat': (['lat'], new_lat),
                    'lon': (['lon'], new_lon)})
    
    # Regrid the data to the new grid 
    regridder = xe.Regridder(ds, ds_out, 'bilinear', reuse_weights=True)
    dr = regridder(ds).to_dataset(name=var)
    
    dr = dr.sel(lat=slice(0,90))

    return dr

In [3]:
file='/home/ubuntu/asr15km.anl.2D.anoms.nc'
anoms=xr.open_dataset(file)

In [4]:
albedos = anoms['ALBEDO']

In [5]:
nt = len(albedos['time'])
# nt = 2

In [6]:
albedosr = []
for t in range(0,nt):
    albedosr.append(regrid(albedos[t], 'ALBEDO'))

albedosr

Reuse existing file: bilinear_720x720_1801x3601.nc


/home/ubuntu/anaconda3/envs/clim680-xesmf/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_720x720_1801x3601.nc
Reuse existing file: bilinear_7

[<xarray.Dataset>
 Dimensions:  (lat: 900, lon: 3601)
 Coordinates:
     time     datetime64[ns] 2000-01-01
     month    int64 ...
   * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9 360.0
   * lat      (lat) float64 0.1 0.2 0.3 0.4 0.5 0.6 ... 89.6 89.7 89.8 89.9 90.0
 Data variables:
     ALBEDO   (lat, lon) float64 0.0 0.0 0.0 0.0 ... -0.02904 -0.02904 -0.02904,
 <xarray.Dataset>
 Dimensions:  (lat: 900, lon: 3601)
 Coordinates:
     time     datetime64[ns] 2000-02-01
     month    int64 ...
   * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9 360.0
   * lat      (lat) float64 0.1 0.2 0.3 0.4 0.5 0.6 ... 89.6 89.7 89.8 89.9 90.0
 Data variables:
     ALBEDO   (lat, lon) float64 0.0 0.0 0.0 ... -0.006142 -0.006142 -0.006142,
 <xarray.Dataset>
 Dimensions:  (lat: 900, lon: 3601)
 Coordinates:
     time     datetime64[ns] 2000-03-01
     month    int64 ...
   * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9 360.0
 

In [8]:
ds_ars = xr.concat(albedosr, dim='time')
ds_ars

<xarray.Dataset>
Dimensions:  (lat: 900, lon: 3601, time: 204)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2016-12-01
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
  * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9 360.0
  * lat      (lat) float64 0.1 0.2 0.3 0.4 0.5 0.6 ... 89.6 89.7 89.8 89.9 90.0
Data variables:
    ALBEDO   (time, lat, lon) float64 0.0 0.0 0.0 ... -0.01976 -0.01976 -0.01976

In [10]:
ds_ars.to_netcdf('/home/ubuntu/asr15km.anl.2D.latlon.anoms.ALBEDO.nc')